## Read full CSV 

In [1]:
import pandas as pd
%pip install music21
from music21 import *
#¡Atencion! Aquí tu path a una carpeta en la que este un archivo MuseScore3.exe
#yo copie el archivo MuseScore3.exe de la carpeta de instalacion a la carpeta en la que tengo el repositorio
path_to_musescore = 'C:/Users/ifc04/OneDrive/Desktop/UNIVERSIDAD/Audio and Music Processing Lab/Lab 3 - Ethnomusicology/GMD-classification/data/'
environment.set('musescoreDirectPNGPath', path_to_musescore)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ifc04\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:

#Read galician_musicxml_full.csv but is in father folder
df = pd.read_csv('../galician_musicxml_full.csv',quotechar='"',skipinitialspace=True)

#df = pd.read_csv('galician_musicxml_full.csv',quotechar='"',skipinitialspace=True)

#drop first column
#name first column id
df = df.rename(columns={'Unnamed: 0': 'id'})
#In this case we dont need the first column because index of row=name of musicxml file
#have in mind that we do need it for the other csv files
df

,id,name,location
0,0,Muiñeira dos Fosados,Fosados
1,1,Detrás das airas,airas
2,2,Muiñeira de Quindous,Quindous
3,3,Muiñeira de Quindous,"Quindous, Cervantes, Lugo"
4,4,Muiñeira,NaN
...,...,...,...
449,449,Carballesa do Incio,Incio
450,450,Muiñeira de Vilarmide,"Vilarmide, A Pontenova"
451,451,Na Ruada,NaN
452,452,Danza Anterga de Lugo,Lugo


## Read file with all the locations

In [3]:

#Read galician_musicxml_full.csv in father folder
df_located = pd.read_csv('../galician_musicxml_located.csv',quotechar='"',skipinitialspace=True)
#df_located = pd.read_csv('galician_musicxml_located.csv',quotechar='"',skipinitialspace=True)
#Rename first column to id
df_located = df_located.rename(columns={df_located.columns[0]: "id"})
"""
Have in mind that for this case the id IS important. ID is the name of the musicxml file
"""
df_located


,id,name,parroquia,concellu,province
0,1,Detrás das airas,NaN,As Airas,Ourense
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo
3,5,Muiñeira de Monforte,NaN,Monforte,Lugo
4,6,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra
...,...,...,...,...,...
255,446,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra
256,449,Carballesa do Incio,NaN,O Incio,Lugo
257,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo
258,452,Danza Anterga de Lugo,NaN,Lugo,Lugo


In [4]:
#Example, let's say we want to load a Muiñeira de Quindous
#Show all the rows where the id is equal to "Muiñeira de Quindous"
df_located.loc[df_located['name'] == 'Muiñeira de Quindous']


,id,name,parroquia,concellu,province
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo


In [5]:
def load_musicxml(index):
    """
    Load a musicxml file and return a music21 stream
    """
    #Set musicxml_file as a file in path data/name and name being string of the index and .musicxml
    #musicxml_file = 'data/' + str(index) + '.musicxml'
    musicxml_file =str(index) + '.musicxml'
    #The files are in the folder data
    #Load the musicxml file
    #print(musicxml_file)
    #try:
    #    s = corpus.parse(musicxml_file)

    #except Exception as e:
        #Get exception if the file is not found
    s=converter.parse(musicxml_file)
    #    return s
    return s

#Let's continue with the example, loading hte second muiñeira that appears:
#muiñeira=load_musicxml(3)

#For loop to 453
#Create dataframe with id, name, key
df_key=pd.DataFrame(columns=['id','name','key','time_signature'])
for i in range(453):
    #get index of df_located
    index=df.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
         
        key=s.analyze('key')
        #Get name of the song in df in this case we can use the index because the index of df is the same as the id
        name=df.loc[i]['name']
        measures0 = s.getElementsByClass(stream.Measure).stream() #numero de compases
        tS = measures0[0].getElementsByClass(meter.TimeSignature).stream()[0]

        time_signature= ''+str(tS.numerator)+'/'+str(tS.denominator)
        df_key.loc[i]=[index,name, key, time_signature]



df_key

,id,name,key,time_signature
0,0,Muiñeira dos Fosados,C major,6/8
1,1,Detrás das airas,C major,6/8
2,2,Muiñeira de Quindous,C major,6/8
3,3,Muiñeira de Quindous,C major,6/8
4,4,Muiñeira,C major,6/8
...,...,...,...,...
448,448,Carballesa,C major,2/4
449,449,Carballesa do Incio,C major,2/4
450,450,Muiñeira de Vilarmide,C major,6/8
451,451,Na Ruada,C major,2/4


In [6]:
df_key_located=pd.DataFrame(columns=['id','name','parroquia','concellu','province','key','time_signature'])
i=0
for i in range(260):
    #get index of df_located
    index=df_located.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
        key=s.analyze('key')
        #Get name of the song in df in this case we can use the index because the index of df is the same as the id
        name=df_located.loc[i]['name']
        parroquia=df_located.loc[i]['parroquia']
        concellu=df_located.loc[i]['concellu']
        province=df_located.loc[i]['province']
            
        measures0 = s.getElementsByClass(stream.Measure).stream() #numero de compases
        tS = measures0[0].getElementsByClass(meter.TimeSignature).stream()[0]
        time_signature= ''+str(tS.numerator)+'/'+str(tS.denominator)
        df_key_located.loc[i]=[index,name,parroquia,concellu,province, key,time_signature]
df_key.drop(df_key.columns[0], axis=1, inplace=True)

df_key

,name,key,time_signature
0,Muiñeira dos Fosados,C major,6/8
1,Detrás das airas,C major,6/8
2,Muiñeira de Quindous,C major,6/8
3,Muiñeira de Quindous,C major,6/8
4,Muiñeira,C major,6/8
...,...,...,...
448,Carballesa,C major,2/4
449,Carballesa do Incio,C major,2/4
450,Muiñeira de Vilarmide,C major,6/8
451,Na Ruada,C major,2/4


In [7]:


df_key_located.drop(df_key_located.columns[0], axis=1, inplace=True)

df_key_located

,name,parroquia,concellu,province,key,time_signature
0,Detrás das airas,NaN,As Airas,Ourense,C major,6/8
1,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8
2,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8
3,Muiñeira de Monforte,NaN,Monforte,Lugo,C major,6/8
4,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra,C major,6/8
...,...,...,...,...,...,...
255,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra,C major,6/8
256,Carballesa do Incio,NaN,O Incio,Lugo,C major,2/4
257,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo,C major,6/8
258,Danza Anterga de Lugo,NaN,Lugo,Lugo,C major,2/4


In [16]:
def ratio_corcheas_negras(index):
    s=load_musicxml(index)
    scoreParts = s.parts.stream()
    my_score = scoreParts[0]
    #print all notes in the score
    measure_list= my_score.getElementsByClass(stream.Measure).stream() 

    #Print all notes
    count_negras=0
    count_corcheas=0
    for measure in measure_list:
        for note in measure.notesAndRests:
            #print(note)
            if not note.isRest:
                
                #Print if the note is a crotchet
                if note.duration.type=='eighth':
                    count_corcheas=count_corcheas+1
                if note.duration.type=='quarter':
                    count_negras=count_negras+1
                #print(note.nameWithOctave)

    #print('Negras: ',count_negras)
    #print('Corcheas: ',count_corcheas)
    #Print count_negras to count_corcheas ratio
    print('Ratio: ',count_negras/count_corcheas)
    return count_negras/count_corcheas
list=[]
for int in range(453):
    val= ratio_corcheas_negras(int)
    list.append(val)



Ratio:  0.09615384615384616
Ratio:  0.2465753424657534
Ratio:  0.3125
Ratio:  0.3076923076923077
Ratio:  0.18
Ratio:  0.24
Ratio:  0.16
Ratio:  0.23076923076923078
Ratio:  0.14102564102564102
Ratio:  0.21888412017167383
Ratio:  0.19658119658119658
Ratio:  0.13043478260869565
Ratio:  0.07017543859649122
Ratio:  0.11494252873563218
Ratio:  0.192090395480226
Ratio:  0.24074074074074073
Ratio:  0.23076923076923078
Ratio:  0.12903225806451613
Ratio:  0.05714285714285714
Ratio:  0.38181818181818183
Ratio:  0.09375
Ratio:  0.21333333333333335
Ratio:  0.38202247191011235
Ratio:  0.25
Ratio:  0.2
Ratio:  0.2727272727272727
Ratio:  0.11194029850746269
Ratio:  0.13333333333333333
Ratio:  0.13333333333333333
Ratio:  0.03773584905660377
Ratio:  0.0380952380952381
Ratio:  0.09375
Ratio:  0.03773584905660377
Ratio:  1.1851851851851851
Ratio:  0.6363636363636364
Ratio:  0.24242424242424243
Ratio:  0.11764705882352941
Ratio:  0.19879518072289157
Ratio:  0.1865671641791045
Ratio:  0.16666666666666666
Ra

In [9]:
#get from df_key_located the one with index 55
name=df_key_located.loc[55]['name']
#print name
print(name)

Muiñeira de San Román de Cervantes


In [17]:
#Both df to csv
#rename first index to id
df_key.to_csv('df_key_ts.csv')
df_key_located.to_csv('df_key_located_ts.csv')